In [2]:
pip install scanpy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import scanpy as sc
import numpy as np
# 737 articles 4613 terms
adata = sc.read("bbcsport.mtx")
data = adata.X
data = data.toarray()
np_data = np.array(data)
transfor_data = []
for i in range(737):
    x = np_data[:,i]
    now = []
    for j in range(len(x)):
        if x[j] > 0:
            now.append(j)
    transfor_data.append(now)                


In [54]:
# Jaccard similarities for all pairs of articles.

def jaccard_sim(t1,t2):
    x = len(t1.intersection(t2))
    y = len(t1.union(t2))
    if y != 0:
        return x / y
    else:
        0

jaccard_withsim = []
jaccard = []
for i in range(737):
    x = set(transfor_data[i])
    for j in range(i+1,737):
        y =set(transfor_data[j])
        if jaccard_sim(x,y) >= 0.5:
            jaccard_withsim.append(((i+1,j+1),jaccard_sim(x,y)))
            jaccard.append((i+1,j+1))
            
            
print("article pairs with similarity >= 0.5")            
for pairs in jaccard_withsim:
    print(f"article{pairs[0][0]} and article{pairs[0][1]} with similarity{pairs[1]}")
        
        
    

article pairs with similarity >= 0.5
article12 and article20 with similarity1.0
article49 and article88 with similarity1.0
article52 and article84 with similarity1.0
article105 and article127 with similarity0.7130044843049327
article118 and article122 with similarity1.0
article124 and article130 with similarity0.9869281045751634
article125 and article133 with similarity1.0
article126 and article134 with similarity1.0
article142 and article215 with similarity0.9511111111111111
article143 and article144 with similarity0.5226130653266332
article153 and article154 with similarity1.0
article161 and article162 with similarity0.9862068965517241
article188 and article189 with similarity1.0
article227 and article487 with similarity0.5905511811023622
article228 and article435 with similarity1.0
article243 and article252 with similarity0.9936908517350158
article312 and article313 with similarity1.0
article357 and article363 with similarity0.5172413793103449
article383 and article386 with similari

In [43]:
import random
# hash functions with a,b,c
N = 50
perms = [ (random.randint(0,737), random.randint(0,737))for i in range(N)]
c = 739

print("hash functions:")
for index,value in enumerate(perms):
    a, b = value
    print(f"hashfunction {index} a:{a}, b:{b}, c:{c}")

# initialize the signature matrix
vec = np.empty([50,737]) 
vec.fill(float('inf'))

# calculate the signature matrix
for i in range(4613):
    l = []
    for index,value in enumerate(perms):
        a, b = value
        l.append((a * i + b) % c)
    for j in range(737):
        if data[i,j] != 0:
            for hashindex in range(50):
                if vec[hashindex,j] > l[hashindex]:
                    vec[hashindex,j] = l[hashindex]

def compare(x,y):
    intersection = 0
    for i in range(50):
        if x[i] == y[i]:
            intersection +=1
    return intersection / 50

# filter the articel pairs with similarity >= 0.5
minhashresult = []
minhashresult_withsim = []
for i in range(737):
    x = vec[:,i]
    for j in range(i+1,737):
        y = vec[:,j]
        s = compare(x,y)
        if s >= 0.5:
            minhashresult_withsim.append(((i+1,j+1),s))
            minhashresult.append((i+1,j+1))
print("\n")
print("article pairs with similarity >= 0.5")
for pairs in minhashresult_withsim:
    print(f"article{pairs[0][0]} and article{pairs[0][1]} with similarity{pairs[1]}")


hash functions:
hashfunction 0 a:271, b:283, c:739
hashfunction 1 a:366, b:733, c:739
hashfunction 2 a:705, b:273, c:739
hashfunction 3 a:539, b:271, c:739
hashfunction 4 a:501, b:367, c:739
hashfunction 5 a:61, b:423, c:739
hashfunction 6 a:464, b:176, c:739
hashfunction 7 a:361, b:659, c:739
hashfunction 8 a:570, b:490, c:739
hashfunction 9 a:621, b:327, c:739
hashfunction 10 a:29, b:133, c:739
hashfunction 11 a:698, b:122, c:739
hashfunction 12 a:165, b:576, c:739
hashfunction 13 a:135, b:267, c:739
hashfunction 14 a:549, b:520, c:739
hashfunction 15 a:11, b:380, c:739
hashfunction 16 a:488, b:586, c:739
hashfunction 17 a:164, b:106, c:739
hashfunction 18 a:566, b:730, c:739
hashfunction 19 a:290, b:285, c:739
hashfunction 20 a:443, b:293, c:739
hashfunction 21 a:311, b:463, c:739
hashfunction 22 a:627, b:619, c:739
hashfunction 23 a:189, b:244, c:739
hashfunction 24 a:662, b:431, c:739
hashfunction 25 a:300, b:555, c:739
hashfunction 26 a:460, b:282, c:739
hashfunction 27 a:460, b:

In [62]:
## false positive and positive negative result with k =50
false_positive = 0
positive_negative = 0

for item in minhashresult:
    if item not in jaccard:
        false_positive += 1
        
print("false positive number:",false_positive)

for item in jaccard:   
    if item not in minhashresult:
        positive_negative += 1
        
print("positive_negative:",positive_negative)    


false positive number: 6
positive_negative: 1


In [66]:
# 100 hash functions
# hash functions with a,b,c
N1 = 100
perms1 = [ (random.randint(0,737), random.randint(0,737))for i in range(N1)]
c1 = 739

print("hash functions:")
for index,value in enumerate(perms1):
    a, b = value
    print(f"hashfunction {index} a:{a}, b:{b}, c:{c}")

# initialize the signature matrix
vec1 = np.empty([100,737]) 
vec1.fill(float('inf'))

# calculate the signature matrix
for i in range(4613):
    l = []
    for index,value in enumerate(perms1):
        a, b = value
        l.append((a * i + b) % c)
    for j in range(737):
        if data[i,j] != 0:
            for hashindex in range(100):
                if vec1[hashindex,j] > l[hashindex]:
                    vec1[hashindex,j] = l[hashindex]

def compare(x,y):
    intersection = 0
    for i in range(100):
        if x[i] == y[i]:
            intersection +=1
    return intersection / 100

# filter the articel pairs with similarity >= 0.5
minhashresult1 = []
minhashresult_withsim1 = []
for i in range(737):
    x = vec1[:,i]
    for j in range(i+1,737):
        y = vec1[:,j]
        s = compare(x,y)
        if s >= 0.5:
            minhashresult_withsim1.append(((i+1,j+1),s))
            minhashresult1.append((i+1,j+1))
print("\n")
print("article pairs with similarity >= 0.5")
for pairs in minhashresult_withsim1:
    print(f"article{pairs[0][0]} and article{pairs[0][1]} with similarity{pairs[1]}")


hash functions:
hashfunction 0 a:76, b:198, c:739
hashfunction 1 a:469, b:352, c:739
hashfunction 2 a:325, b:241, c:739
hashfunction 3 a:676, b:159, c:739
hashfunction 4 a:659, b:450, c:739
hashfunction 5 a:15, b:388, c:739
hashfunction 6 a:180, b:561, c:739
hashfunction 7 a:210, b:212, c:739
hashfunction 8 a:150, b:140, c:739
hashfunction 9 a:574, b:653, c:739
hashfunction 10 a:697, b:737, c:739
hashfunction 11 a:668, b:533, c:739
hashfunction 12 a:689, b:89, c:739
hashfunction 13 a:176, b:535, c:739
hashfunction 14 a:112, b:216, c:739
hashfunction 15 a:569, b:231, c:739
hashfunction 16 a:602, b:516, c:739
hashfunction 17 a:110, b:244, c:739
hashfunction 18 a:297, b:540, c:739
hashfunction 19 a:94, b:628, c:739
hashfunction 20 a:693, b:613, c:739
hashfunction 21 a:304, b:42, c:739
hashfunction 22 a:424, b:357, c:739
hashfunction 23 a:624, b:493, c:739
hashfunction 24 a:397, b:4, c:739
hashfunction 25 a:684, b:266, c:739
hashfunction 26 a:537, b:560, c:739
hashfunction 27 a:631, b:452,

In [67]:
## false positive and positive negative result with k = 100
false_positive1 = 0
positive_negative1 = 0

for item in minhashresult1:
    if item not in jaccard:
        false_positive1 += 1
        
print("false positive number:",false_positive1)

for item in jaccard:   
    if item not in minhashresult1:
        positive_negative1 += 1
        
print("positive negative number:",positive_negative1)    


false positive number: 3
positive_negative: 0
